# 使用 Hugo 制作文档网站

v20220601-1

![url](https://s3.laisky.com/uploads/2022/05/hugo.png)

简单介绍一下使用 Hugo 搭建网站的基础用法。

以及 XSS 现阶段的实践。

我也是第一次使用 Hugo，所以还算不上“最佳实践”，只能算抛砖引玉，方便快速上手。

## Changelog

| Version | Description |
| :-- | :-- |
| v20220601-1 | 第一版 |

## 体验

先有个直观的感受，可以打开 XSS 目前的站点体验下。

<http://basebit.laisky.com/>

（域名还没申请下来， 暂时先用我的个人域名）

### 安装

可以参考文档中的 【工具集 -> Hugo -> 安装】，这里不再赘述

<http://basebit.laisky.com/doc/xss/2022/05/hugo/#install>

## QuickStart

完成安装、初始化、选择主题后，会得到如下的目录结构：

![url](https://s3.laisky.com/uploads/2022/05/hugo-dir.png)

需要关心的就三个：

1. `content/`：防止 markdown 文件的地方
1. `public/`：自动生成的，可供发布的站点静态文件
1. `config.toml`：配置文件

其他一些可能用的：

1. `static/`：防止自定义静态文件
1. `themes/`：样式源码，魔改的话可以改这里

### Theme

`config.toml`

![config](https://s3.laisky.com/uploads/2022/05/hugo-config.png)

实际上 theme 就是从 `/themes` 下面找相对应的文件夹。


初始化时安装 theme：

```sh
git submodule add https://github.com/Laisky/hugo-book themes/hugo-book
```

我 fork 了 `alex-shpak/hugo-book`，增加了对 `custom_css` 和 `custom_js` 的支持。

```toml
[params]
  custom_css = ["css/custom.css"]
  custom_js = ["js/custom.js"]
```

相对应的文件在：

- `/static/css/custom.css`
- `/static/js/custom.js`


### Samples

可以在文档中找到很多样式的例子：<https://hugo-book-demo.netlify.app/docs/shortcodes/details/>


![sample](https://s3.laisky.com/uploads/2022/06/hugo-sample.png)

### Content

```
content
├── _index.md
├── research
│   ├── _index.md
│   ├── papers
│   ├── secure_vm
│   └── sgx
├── terms.md
├── tools
│   ├── _index.md
│   └── hugo.md
└── xss
    ├── _index.md
    └── tee.md
```

`_index.md` 相当于 `index.html`，扮演着主页的角色。

因为设置了默认 Menu，所以文件结构就是目录结构。

因为默认 URL 是通过文件路径来生成的，这个可能会经常变动，
做外链的话会很不方便。
所以最好在页面属性中，手动指定一个不变的固定 URL。

![page](https://s3.laisky.com/uploads/2022/05/hugo-page.png)

可以看到最终生效的就是手动指定的 URL，而且自动加上了 `url prefix`

![page-url](https://s3.laisky.com/uploads/2022/06/hugo-page-url.png)


### Link

站内连接需要用模版语法 

`{{% ref "<页面绝对路径>" %}}`

这个页面地址指的是 markdown 文件的路径，而不是手动在页面属性中指定的固定 URL。
这点确实很不方便，如果调换文件路径的话，需要记得批量修改一下。

```markdown
[QE]({{% ref "/terms#intel-sgx-qe" %}}) 用来对 [QUOTE]({{% ref "/terms#intel-sgx-quote" %}}) 签名的非对称密钥。
```

我写了个 python 小脚本，可以扫描检查 link 的指向和描述是否统一

```markdown
make lint
```

![hugo-lint](https://s3.laisky.com/uploads/2022/06/hugo-lint.png)

如果是不同的文字关联到相同的 URL，就不会报错，而只是打一行 warning

### Sitemap

```toml
[sitemap]
  changefreq = 'daily'
  filename = 'sitemap.xml'
```

就可以在 `/sitemap.xml` 生成 sitemap 文件，日后做全文索引的话，通过 sitemap 可以较为方便的爬取所有页面。

#### autolinker

本来想写一个自动生成链接的脚本，但是比预想的复杂，需要考虑标题行、括号、代码块、超链接等太多因素，目前只有一个很简单的实现，而且存在较多 BUG。
原以为简单搞个正则就行，目前看来还是得用 AST。



日后有空再写吧……

![pigeon](https://s3.laisky.com/uploads/2022/06/pigeon.jpg)

## 静态文件

如何存放静态文件，有三个方案：

1. 放在 hugo 项目文件夹的 `/static` 内
2. 放在文档服务器的 `/var/www/static` 内
3. 放在内网的 S3 服务器内

### Hugo Static

Hugo 项目文件夹的 `/static` 文件夹就是用来放静态文件的。

这个方案的缺点是会用 git 去追踪静态文件，这样比较重。

一般来说 `/static` 里还是只放一些静态的 css、js、icon 之类的小文件。

### Nginx

自行把文件通过 `scp/rsync` 上传到服务器指定目录，然后通过 Nginx 对外暴露。

![static](https://s1.laisky.com/uploads/2022/06/doc-static.png)



推荐用 rsync，可以防止文件被覆盖 `rsync --ignore-existing -raz --progress <FROM> <TO>`

文件夹按照 `/<year>/<month>` 的形式拆分

### S3

在服务器上用 seaweedfs 搭了一个 S3，通过浏览器插件可以实现很方便的查阅和上传功能

<http://s3.basebit.laisky.com/>

![s3](https://s3.laisky.com/uploads/2022/06/doc-s3.png)

缺点是权限不太好控制，一旦放开，任何人都可以编辑。
配置个 basic auth 的话，目前的 S3 client 都支持不太好。

## 愿景

主站（目前暂时是 `http://basebbit.laisky.com`）作为 XEGO 的总门户，制作一个索引功能的 landing page，包含指向各个主要 repo、文档、工具集的链接。

目前 XSS Doc 仅占用了 `/doc/xss/` 这个子路径。其他组的文档站在部署好后，也可以加入到相对应的子路径里。

TODO List：

1. 一个简单的 landing page
1. 全文索引和搜索

关于文档的一些其他补充

这次仅介绍了脱离于代码文件的独立文档。
这种文档一般用于维护一些综述性质的文档。

其他一些和代码具有强关联的文档，建议还是放在代码所在的地方，然后按照所选语言的常见规范来直接从代码注释生成文档。

比如 python 可以用 readthedoc 等，go 可以用 godoc 等。

我个人认为文档的具体形式不重要，重要的是要通过链接和搜索关联到一个统一的站点网络内。

# THANKS